In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp /content/gdrive/MyDrive/Face_23C.zip /content/
!unzip --qq Face_23C.zip

In [ ]:
!pip install tensorflow_hub
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, Model
from tensorflow import keras
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os


In [ ]:
data_root = "/content/Face_23C"

In [ ]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)

datagen_kwargs = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    TRAINING_DATA_DIR, 
    subset="validation", 
    shuffle=True,
    target_size=IMAGE_SHAPE
)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR, 
    subset="training", 
    shuffle=True,
    target_size=IMAGE_SHAPE)

In [ ]:
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

In [ ]:
Base_model = InceptionV3(weights = 'imagenet',
                          include_top = False,
                          input_shape = (224, 224, 3),
                          )
# Base_model = ResNet50V2(weights = 'imagenet',
#                           include_top = False,
#                           input_shape = (224, 224, 3),
#                           )
# Base_model = VGG16(weights = 'imagenet',
#                           include_top = False,
#                           input_shape = (224, 224, 3),
#                           )
                   

In [ ]:
model = tf.keras.models.Sequential([
                                    Base_model,
                                    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(256, activation='relu'),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(64, activation='relu'),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(32, activation='relu'),
                                    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
                                    
                                    
])
model.summary()

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = 'categorical_crossentropy', 
    metrics = ['acc'])


In [ ]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = model.fit(
    train_generator,
    epochs=300,
    verbose=1,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=val_steps_per_epoch).history

In [ ]:
final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)

In [ ]:
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

In [ ]:

plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
#plt.ylim([0,2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])
plt.legend(['Training loss', 'Validation loss'], loc='best')
plt.show()

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])
plt.legend(['Training accuracy', 'Validation accuracy'], loc='best')
plt.show()

In [ ]:
FACE_SAVED_MODEL = "saved_models/face_23P"
tf.saved_model.save(model, FACE_SAVED_MODEL)

INFO:tensorflow:Assets written to: saved_models/face_23P/assets


In [ ]:
!pip install tensorflow_hub
import tensorflow_hub as hub
import numpy as np
import os

In [ ]:
face_model = hub.load(FACE_SAVED_MODEL)
print(face_model)

In [ ]:
!mkdir "tflite_models"


In [ ]:
TFLITE_MODEL = "tflite_models/face.tflite"

In [ ]:
# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : face_model(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)

# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)